In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass

import emcee
from scipy.optimize import minimize
from scipy.stats import t, norm

In [ ]:
from surp import subgiants
import surp
import vice
from surp import gce_math as gcem
import arya

import sys
sys.path.append("../..")


In [ ]:
from singlezone import run_singlezone

In [ ]:
def calc_model(**params):
    sz, h = run_singlezone(**params)
    
    return h.O_FE, h.C_O, h.sfr

In [ ]:
def plot_sz(label="", **kwargs,):
    x, y, w = calc_model(**kwargs)
    plt.scatter(x, y, s=w, label=label)
    plt.xlabel("[Mg/Fe]")
    plt.ylabel("[C/Mg]")
    arya.Legend(loc=-1, color_only=True)

# Simple modeling

## AGB Yield properties

In [ ]:
params = surp.YieldParams.from_file("../models/fiducial/yield_params.toml")

In [ ]:
for model in surp.AGB_MODELS:
    surp.set_yields(params, Y_c_agb=model, alpha_c_agb=1, verbose=False)
    plot_sz(label=model)

In [ ]:
def set_ana_yields(**kwargs):
    surp.set_yields(Y_c_agb="A", kwargs_c_agb=kwargs, verbose=False)

In [ ]:
def set_yields(**kwargs):
    surp.set_yields(verbose=false, **kwargs)

In [ ]:
set_yields()
plot_sz(label="fiducial")
set_ana_yields(zeta=0, t_D=0.2)
plot_sz(label="analytic")

In [ ]:
Y0 = 0.002

In [ ]:
for model in [0, 0.1, 0.2, 0.3]:
    set_ana_yields(zeta=0, y0 = Y0 * model)
    plot_sz(label=f"f_agb={model}")

In [ ]:
for model in [0, 0.2, 0.5, 1]:
    set_ana_yields(zeta=0, t_D=model)
    plot_sz(label=f"t_D={model}")

In [ ]:
for model in [0.3, 0.5, 0.8, 2]:
    set_ana_yields(zeta=0, tau_agb=model)
    plot_sz(label=f"tau_agb = {model}")

In [ ]:
for model in [-0.04, -0.02, 0, 0.02]:
    set_ana_yields(zeta=model)
    plot_sz(label=f"zeta_agb = {model}")
    

## CCSNe

In [ ]:
for model in [-0.05, -0.02, 0, 0.001]:
    make_yields(zeta_cc=model, agb_model="A", zeta_agb=0)
    plot_sz(label=f"zeta_cc = {model}")
    

In [ ]:
make_yield_params(y_c_cc="BiLogLin")

In [ ]:
for model in ["Lin", "LogLin",  "BiLogLin", "Quadratic"]:
    make_yields(y_c_cc=model, verbose=True)
    plot_sz(label=f"cc model = {model}")

plt.ylim(-1)

## Iron

In [ ]:
for model in [0.8, 1, 1.2]:
    make_yields(fe_ia_factor=model)
    plot_sz(label=f"fe_ia_enhancement = {model}")
    

In [ ]:
def dtd_1(t):
    return np.where(t > 0.15, t**(-1.1), 0)

def dtd_2(t):
    return np.where(t > 0.15, np.exp(-(t-0.15)), 0)

t = np.logspace(-1.3, np.log10(13), 1000)
dt = np.gradient(t)
R1 = np.cumsum(dtd_1(t)*dt)
R2 = np.cumsum(dtd_2(t)*dt)

plt.plot(t, R1 / R1[-1], label="plaw")
plt.plot(t, R2 / R2[-1], label="exp", ls="--")

plt.xscale("log")
plt.xlabel("time / Gyr")
plt.ylabel("R(t) / R(today) for SneIa ")
plt.legend()

In [ ]:
for model in ["plaw", "exp"]:
    plot_sz(label=f"fe_dtd = {model}", RIa=model)
    

## GCE Parameters

In [ ]:
for model in [1, 2, 4, 10, 20]:
    plot_sz(label=f"tau_star = {model}", tau_star=model)
    

In [ ]:
for model in [1, 5, 14, 30]:
    plot_sz(label=f"tau_sfh = {model}", tau_sfh=model)
    

In [ ]:
for model in ["sfr", "ifr", "gas"]:
    plot_sz(label=f"mode = {model}", mode=model)
    

In [ ]:
plot_sz(label=f"insideout", sfh=surp.simulation.sfh_models.insideout(tau_sfh=14))
plot_sz(label=f"linex", sfh=surp.simulation.sfh_models.linexp(tau_sfh=14))
plot_sz(label=f"const", sfh=lambda x: 1)
plot_sz(label="exp")


## Bimodality

In [ ]:
t12 = 7

In [ ]:
kwargs = dict()
sz1, h1 = run_singlezone(**kwargs)
sz2, h2 = run_singlezone(tau_sfh=3, tau_star=0.7, eta=0, t_end=t12, RIa="exp", **kwargs)

filt = np.array(h1["time"]) > t12 - 2
h1 = h1[filt]

In [ ]:

plt.plot(h1["[fe/h]"], h1.MG_FE)

plt.plot(h2["[fe/h]"], h2.MG_FE)

plt.hist2d(subgiants.FE_H, subgiants.MG_FE, 300, cmap="Greys", norm="log");
plt.xlim(-1, 0.5)
plt.ylim(-0.1, 0.5)
plt.xlabel("[Fe/H]")
plt.ylabel("[Mg/Fe]")
plt.show()


plt.plot(h1.MG_FE, h1.C_MG)
plt.plot(h2.MG_FE, h2.C_MG)
plt.hist2d(subgiants.MG_FE, subgiants.C_MG, 300, cmap="Greys", norm="log");
plt.xlim(-0.2, 0.6)
plt.ylim(-0.5, 0.1)